Import required elements

In [1]:
from src.keras_model.lenet5_keras_model import LeNet5
from src.datasets.dataset_generators import DatasetGenerators
from keras.losses import categorical_crossentropy

Using TensorFlow backend.


Setup basic params

In [2]:
dataset_dir = '/Users/igor/Python/CNN-Architecture-Comparison/data'
dataset_name = 'cifar10'
dataset_image_size = (32, 32)
dataset_grayscale = True
model_optimizer = 'adam'
model_loss = categorical_crossentropy
model_metrics = ['accuracy']
fit_epochs = 10
evaluation_bath_size = 128

Load dataset (CIFAR10)

In [3]:
dataset_generators = DatasetGenerators(
    name=dataset_name,
    image_size=dataset_image_size,
    dataset_dir=dataset_dir,
    grayscale=dataset_grayscale
)

Loading train set for cifar10...
Found 35000 images belonging to 10 classes.
Loading test set for cifar10...
Found 10000 images belonging to 10 classes.
Loading validation set for cifar10...
Found 5000 images belonging to 10 classes.


Create & compile model

In [4]:
model = LeNet5(
    classes=dataset_generators.test.num_classes
).getModel()

model.compile(
    optimizer=model_optimizer,
    loss=model_loss,
    metrics=model_metrics
)

Perform epoch learning

In [ ]:
learn_history = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test
)

Epoch 1/10
1094/1094 [==============================] - 13s 11ms/step - loss: 1.9266 - accuracy: 0.3614 - val_loss: 1.1191 - val_accuracy: 0.4416
Epoch 2/10
1094/1094 [==============================] - 13s 11ms/step - loss: 1.4686 - accuracy: 0.4875 - val_loss: 1.4817 - val_accuracy: 0.4906
Epoch 3/10
1094/1094 [==============================] - 13s 11ms/step - loss: 1.3475 - accuracy: 0.5302 - val_loss: 0.8732 - val_accuracy: 0.5331
Epoch 4/10
1094/1094 [==============================] - 12s 11ms/step - loss: 1.2521 - accuracy: 0.5608 - val_loss: 1.4710 - val_accuracy: 0.5179
Epoch 5/10
1094/1094 [==============================] - 13s 12ms/step - loss: 1.1628 - accuracy: 0.5958 - val_loss: 1.1755 - val_accuracy: 0.5577
Epoch 6/10
1094/1094 [==============================] - 13s 12ms/step - loss: 1.0866 - accuracy: 0.6222 - val_loss: 1.0707 - val_accuracy: 0.5553
Epoch 7/10
1094/1094 [==============================] - 12s 11ms/step - loss: 1.0176 - accuracy: 0.6474 - val_loss: 2.0849 -

Evaluate model

In [ ]:
model.evaluate_generator(
    generator=dataset_generators.validate
)
